In [ ]:
import ipywidgets as widgets
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt, dates

%matplotlib inline
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/PittChallengeCode/
 

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/PittChallengeCode/'
/content/drive/MyDrive/PittChallengeCode


In [ ]:
cgm = pd.read_csv('./Data Tables/HDeviceCGM.txt', sep = '|')
medication = pd.read_csv('./Data Tables/HMedication.txt', sep = '|')
insulin = pd.read_csv('./Data Tables/HInsulin.txt', sep = '|')



In [ ]:

def get_patient_cgm(patient_id, data = cgm):
  idx = cgm['PtID'] == patient_id
  patient_table_cgm = cgm[idx]
  unique_days = patient_table_cgm['DeviceDtTmDaysFromEnroll'].unique()
  day1 = patient_table_cgm['DeviceDtTmDaysFromEnroll'] == unique_days[0]
  time = pd.to_datetime(patient_table_cgm[day1]['DeviceTm']).to_numpy()
  time_srt = time.argsort()
  gvals = patient_table_cgm[day1]['GlucoseValue'].to_numpy()
  time = time[time_srt]
  gvals = gvals[time_srt]
  return time, gvals

def get_patient_route(patient_id, data = insulin):
  idx = insulin['PtID'] == patient_id
  patient_table_insulin = insulin[idx]
  route = patient_table_insulin['InsRoute'].to_numpy()
  return route

def get_patient_meds(patient_id, data = medication):
  idx = medication['PtID'] == patient_id
  patient_table_med = medication[idx]
  med_lst = patient_table_med['DrugName'].to_numpy()
  return med_lst

patient_id = 183
x , y = get_patient_cgm(patient_id)

patient_last_gv = y[-1]

route = get_patient_route(patient_id)
meds = get_patient_meds(patient_id)

In [ ]:

patient_id = 184
x , y = get_patient_cgm(patient_id)

patient_last_gv = y[-1]

route = get_patient_route(patient_id)
meds = get_patient_meds(patient_id)

w_header = widgets.HTML(f'<h2>header</h2>')
# w_l1 = widgets.HTML(f'<h1>Patient Name : {str(patient_id)}</h1>')
w_l1 = widgets.widgets.Text(value='183', placeholder='Type something',
                    description='Patient Name', disabled=False, style = {'font-size':'35px'})
w_l2 = widgets.HTML(f'<h1>{patient_last_gv} mg/dL</h1>')
w_text = widgets.Textarea(placeholder='Write something!', layout=widgets.Layout(width = '80%'))
w_button = widgets.Button(description='Run')
w_output = widgets.Output()
 
with w_output:
    fig, ax = plt.subplots(figsize=(10, 10)) #fontsize="xx-large", fontweight="bold"
    ax.plot(x,y)
    plt.ylabel('mg/dL')
    plt.xlabel('Time')
    ax.xaxis.set_major_formatter(dates.DateFormatter('%H:%M'))
    plt.show(fig)
w_left = widgets.VBox([w_l1, w_l2, w_output], layout=widgets.Layout(align_items='center'))

r_l1 = widgets.Text(value='Unit/Kg/day', placeholder='Type something',
                    description='Insulin Dose', disabled=False)
r_l2 = widgets.Text(value=route[0], 
                    description='Route', disabled=False)

r_l3 = widgets.Dropdown(options=meds.tolist(), value=meds.tolist()[0], 
                        description='Co-med', disabled=False)
r_l4 = widgets.Text(value='mg', placeholder='Type something',
                    description='Co-med dose', disabled=False)
display_text = widgets.HTML('<p style="color:blue;text-align:center;size:25">click run</p>')

w_right = widgets.VBox([r_l1,r_l2, r_l3, r_l4, w_button, display_text], layout=widgets.Layout(align_items='center', align_contents='center'))


w_ui = widgets.AppLayout(#header=w_header,
          left_sidebar=w_left,
          # center=center,
          right_sidebar = w_right,
          # footer=footer,
          )

def plt_func(c):
  w_output.clear_output()
  patient_id = int(c.value)
  x , y = get_patient_cgm(patient_id)

  with w_output:
    fig, ax = plt.subplots(figsize=(10, 10)) #fontsize="xx-large", fontweight="bold"
    ax.plot(x,y)
    plt.ylabel('mg/dL')
    plt.xlabel('Time')
    ax.xaxis.set_major_formatter(dates.DateFormatter('%H:%M'))
    plt.show(fig)
  
  patient_last_gv = y[-1]
  route = get_patient_route(patient_id)
  meds = get_patient_meds(patient_id)
  r_l2.value = route[0]
  r_l3.options = meds.tolist()

  w_l2.value = f'<h1>{patient_last_gv} mg/dL</h1>'
# w_ui = widgets.VBox([w_header, w_text, w_button, w_output], layout=widgets.Layout(align_items='center'))

def on_button_clicked(c):
  try:
    change_val = float(r_l1.value )*1.3
    txt = f'<h2> Result </h2>\n <h3>Dose change needed {change_val} Unit/Kg/day </h3> <p> Dosing recommendation: Insulin doses should be increased by at least 30% when the corticosteroid therapy is started and may need to be increased by 70% or more to normalize blood glucose levels. Increased insulin administered above the 30% increase needs to be individualized (Bevier et al.) </p>'
    display_text.value = txt
  except ValueError:
    txt ='Please Insert a valid Insulin Dose value '
    display_text.value = txt
w_l1.on_submit(plt_func)
w_button.on_click(on_button_clicked)

display(w_ui)

AppLayout(children=(VBox(children=(Text(value='183', description='Patient Name', placeholder='Type something')…